# Run learners in job scripts

## Define the learners

We need the following variables:
* `learners` a list of learners
* `combos` a list of dicts of parameters that describe each learner
* `fnames` a list of filenames of each learner

In [ ]:
%%writefile _learners.py

import adaptive
from functools import partial


def h(x, offset=0):
    import numpy as np
    import random

    # Burn some CPU time just because
    for _ in range(10):
        np.linalg.eig(np.random.rand(1000, 1000))

    a = 0.01
    return x + a ** 2 / (a ** 2 + (x - offset) ** 2)


offset = [i / 100 for i in range(100)]

combos = adaptive.utils.named_product(offset=offset)

learners = []
fnames = []

folder = "data/"

for i, combo in enumerate(combos):
    f = partial(h, offset=combo["offset"])
    learner = adaptive.Learner1D(f, bounds=(-1, 1))
    fnames.append(f"{folder}{combo}")
    learners.append(learner)

learner = adaptive.BalancingLearner(learners)

In [ ]:
# Execute the previous code block and plot the learners
from _learners import *
import adaptive
adaptive.notebook_extension()
learner.load(fnames)
learner.plot()

## The Python script that is being run in the job

In [ ]:
# Make sure to use the headnode's in the next cell
from adaptive_scheduler import server_support
url = server_support.get_allowed_url()
print(url)

In [ ]:
%%writefile run_learner.py

import adaptive
from mpi4py.futures import MPIPoolExecutor

from adaptive_scheduler import client_support

url = "tcp://10.75.0.5:37371"

if __name__ == "__main__":
    learner, fname = client_support.get_learner(url)
    learner.load(fname)
    ex = MPIPoolExecutor()
    runner = adaptive.Runner(
        learner,
        executor=ex,
        goal=None,
        shutdown_executor=True,
        retries=10,
        raise_if_retries_exceeded=False,
    )
    runner.start_periodic_saving(dict(fname=fname), interval=600)
    runner.ioloop.run_until_complete(runner.task)  # wait until runner goal reached
    client_support.is_done(url, fname)

# Import the files that were created

In [ ]:
import asyncio
from importlib import reload

from adaptive_scheduler import server_support
from pprint import pprint
from tinydb import TinyDB

import _learners, run_learner

reload(_learners)
reload(run_learner)

db_fname = 'running.tinydb'

In [ ]:
# Create a new database
server_support.create_empty_db(db_fname, _learners.fnames, _learners.combos)

## Check the running learners
All the onces that are `None` are still `PENDING` or are not scheduled.

In [ ]:
with TinyDB(db_fname) as db:
    pprint(db.all()[:10])

## Start the job scripts

In [ ]:
# Get some unique names for the jobs
from run_learner import url
job_names = [f"test-job-{i}" for i in range(len(_learners.learners))]

ioloop = asyncio.get_event_loop()

database_task = ioloop.create_task(
    server_support.manage_database(url, db_fname)
)

job_task = ioloop.create_task(
    server_support.manage_jobs(job_names, db_fname, ioloop, cores=50*8, interval=60)
)

In [ ]:
job_task.print_stack()

In [ ]:
database_task.print_stack()

In [ ]:
# Run this to STOP managing the database and jobs
# job_task.cancel(), database_task.cancel()